# 토픽 모델링(Topic Modeling)
#### 토픽(Topic)은 한국어로는 주제라고 한다. 
#### 토픽 모델링(Topic Modeling)이란 기계 학습 및 자연어 처리 분야에서 토픽이라는 문서 집합의 추상적인 주제를 발견하기 위한 통계적 모델 중 하나로, 

#### 텍스트 본문의 숨겨진 의미 구조를 발견하기 위해 사용되는 텍스트 마이닝 기법이다.

## [1] TF-IDF(Term Frequency-Inverse Document Frequency) : 단어 빈도-역 문서 빈도
TF-IDF(Term Frequency - Inverse Document Frequency)는 정보 검색과 텍스트 마이닝에서 이용하는 가중치로, 여러 문서로 이루어진 문서군이 있을 때 어떤 단어가 특정 문서 내에서 얼마나 중요한 것인지를 나타내는 통계적 수치이다. 문서의 핵심어를 추출하거나, 검색 엔진에서 검색 결과의 순위를 결정하거나, 문서들 사이의 비슷한 정도를 구하는 등의 용도로 사용할 수 있다.

In [1]:
import pandas as pd # 데이터프레임 사용을 위해
import numpy as np
from math import log # IDF 계산을 위해

In [2]:
docs = [
  '먹고 싶은 사과',
  '먹고 싶은 바나나',
  '길고 노란 바나나 바나나',
  '저는 과일이 좋아요'
] 
vocab = list(set(w for doc in docs for w in doc.split()))
vocab.sort()
vocab

['과일이', '길고', '노란', '먹고', '바나나', '사과', '싶은', '저는', '좋아요']

In [3]:
N = len(docs) # 총 문서의 수 , 4

def tf(t, d):  # 특정 문서 d에서의 특정 단어 t의 등장 횟수
    return d.count(t)

def idf(t):  # df(t):특정 단어 t가 등장한 문서의 수,idf(t):df(t)의 역수, log는 N이 커지면 값이 기하급수로 늘어나는걸 방지,분모 1은 df가 0일 경우 대비
    df = 0
    for doc in docs:
        df += t in doc
    return log(N/(df + 1))

def tfidf(t, d): # tf(t,d)와 idf(t)를 곱한 값
    return tf(t,d)* idf(t)

In [4]:
# Document-Term Matrix(DTM) : 문서에서 등장하는 각 단어들의 빈도를 행렬로 표현
result = []
for i in range(N):
    result.append([])
    d = docs[i]
    for j in range(len(vocab)):
        t = vocab[j]        
        result[-1].append(tf(t, d))

tf_ = pd.DataFrame(result, columns = vocab)
tf_

,과일이,길고,노란,먹고,바나나,사과,싶은,저는,좋아요
0,0,0,0,1,0,1,1,0,0
1,0,0,0,1,1,0,1,0,0
2,0,1,1,0,2,0,0,0,0
3,1,0,0,0,0,0,0,1,1


In [5]:
result = []
for j in range(len(vocab)):
    t = vocab[j]
    result.append(idf(t))

idf_ = pd.DataFrame(result, index = vocab, columns = ["IDF"])
idf_

,IDF
과일이,0.693147
길고,0.693147
노란,0.693147
먹고,0.287682
바나나,0.287682
사과,0.693147
싶은,0.287682
저는,0.693147
좋아요,0.693147


In [6]:
result = []
for i in range(N):
    result.append([])
    d = docs[i]
    for j in range(len(vocab)):
        t = vocab[j]

        result[-1].append(tfidf(t,d))

tfidf_ = pd.DataFrame(result, columns = vocab)
tfidf_

# TF-IDF는 모든 문서에서 자주 등장하는 단어는 중요도가 낮다고 판단하며, 특정 문서에서만 자주 등장하는 단어는 중요도가 높다고 판단한다 
# TF-IDF 값이 낮으면 중요도가 낮은 것이며, TF-IDF 값이 크면 중요도가 큰 것. 즉, the나 a와 같이 불용어의 경우에는 모든 문서에 자주 
# 등장하기 마련이기 때문에 자연스럽게 불용어의 TF-IDF의 값은 다른 단어의 TF-IDF에 비해서 낮아지게 된다

,과일이,길고,노란,먹고,바나나,사과,싶은,저는,좋아요
0,0.000000,0.000000,0.000000,0.287682,0.000000,0.693147,0.287682,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.287682,0.287682,0.000000,0.287682,0.000000,0.000000
2,0.000000,0.693147,0.693147,0.000000,0.575364,0.000000,0.000000,0.000000,0.000000
3,0.693147,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.693147,0.693147


### 사이킷런을 이용한 DTM과 TF-IDF

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
corpus = [
    'you know I want your love',
    'I like you',
    'what should I do ',    
]
vector = CountVectorizer()
print(vector.fit_transform(corpus).toarray()) # 코퍼스로부터 각 단어의 빈도 수를 기록한다.
print(vector.vocabulary_)                     # 각 단어의 인덱스가 어떻게 부여되었는지를 보여준다 , 한개의 문자는 인덱스를 만들지 않음

[[0 1 0 1 0 1 0 1 1]
 [0 0 1 0 0 0 0 1 0]
 [1 0 0 0 1 0 1 0 0]]
{'you': 7, 'know': 1, 'want': 5, 'your': 8, 'love': 3, 'like': 2, 'what': 6, 'should': 4, 'do': 0}


In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
corpus = [
    'you know I want your love',
    'I like you',
    'what should I do ',    
]
tfidfv = TfidfVectorizer().fit(corpus)
print(tfidfv.transform(corpus).toarray())
print(tfidfv.vocabulary_)

[[0.         0.46735098 0.         0.46735098 0.         0.46735098
  0.         0.35543247 0.46735098]
 [0.         0.         0.79596054 0.         0.         0.
  0.         0.60534851 0.        ]
 [0.57735027 0.         0.         0.         0.57735027 0.
  0.57735027 0.         0.        ]]
{'you': 7, 'know': 1, 'want': 5, 'your': 8, 'love': 3, 'like': 2, 'what': 6, 'should': 4, 'do': 0}


## [2] 잠재 의미 분석(Latent Semantic Analysis, LSA)
SA는 기본적으로 DTM이나 TF-IDF 행렬에 절단된 SVD(truncated SVD)를 사용하여 차원을 축소시키고, 단어들의 잠재적인 의미를 끌어낸다

In [9]:
from sklearn.datasets import fetch_20newsgroups

In [10]:
# wenty Newsgroups 뉴스 그룹 데이터 읽어오기 
dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
documents = dataset.data   # list
len(documents) # 11314

11314

In [11]:
documents[1]

"\n\n\n\n\n\n\nYeah, do you expect people to read the FAQ, etc. and actually accept hard\natheism?  No, you need a little leap of faith, Jimmy.  Your logic runs out\nof steam!\n\n\n\n\n\n\n\nJim,\n\nSorry I can't pity you, Jim.  And I'm sorry that you have these feelings of\ndenial about the faith you need to get by.  Oh well, just pretend that it will\nall end happily ever after anyway.  Maybe if you start a new newsgroup,\nalt.atheist.hard, you won't be bummin' so much?\n\n\n\n\n\n\nBye-Bye, Big Jim.  Don't forget your Flintstone's Chewables!  :) \n--\nBake Timmons, III"

In [12]:
print(dataset.target_names)

['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


### 텍스트 전처리

In [13]:
news_df = pd.DataFrame({'document':documents})
# 특수 문자 제거
news_df['clean_doc'] = news_df['document'].str.replace("[^a-zA-Z]", " ")
# 길이가 3이하인 단어는 제거 (길이가 짧은 단어 제거)
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
# 전체 단어에 대한 소문자 변환
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: x.lower())

In [14]:
news_df['clean_doc'][1]

'yeah expect people read actually accept hard atheism need little leap faith jimmy your logic runs steam sorry pity sorry that have these feelings denial about faith need well just pretend that will happily ever after anyway maybe start newsgroup atheist hard bummin much forget your flintstone chewables bake timmons'

In [15]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\IPA(백찬휘)\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\IPA(백찬휘)\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [16]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english') # NLTK로부터 불용어를 받아옵니다.
tokenized_doc = news_df['clean_doc'].apply(lambda x: x.split()) # 토큰화
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])
# 불용어를 제거

In [17]:
print(tokenized_doc[1])

['yeah', 'expect', 'people', 'read', 'actually', 'accept', 'hard', 'atheism', 'need', 'little', 'leap', 'faith', 'jimmy', 'logic', 'runs', 'steam', 'sorry', 'pity', 'sorry', 'feelings', 'denial', 'faith', 'need', 'well', 'pretend', 'happily', 'ever', 'anyway', 'maybe', 'start', 'newsgroup', 'atheist', 'hard', 'bummin', 'much', 'forget', 'flintstone', 'chewables', 'bake', 'timmons']


### TF-IDF 행렬 만들기

In [18]:
# 역토큰화 (토큰화 작업을 역으로 되돌림)
detokenized_doc = []
for i in range(len(news_df)):
    t = ' '.join(tokenized_doc[i])
    detokenized_doc.append(t)

news_df['clean_doc'] = detokenized_doc

In [19]:
news_df['clean_doc'][1]

'yeah expect people read actually accept hard atheism need little leap faith jimmy logic runs steam sorry pity sorry feelings denial faith need well pretend happily ever anyway maybe start newsgroup atheist hard bummin much forget flintstone chewables bake timmons'

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words='english', 
max_features= 1000, # 상위 1,000개의 단어를 보존 
max_df = 0.5, 
smooth_idf=True)

X = vectorizer.fit_transform(news_df['clean_doc'])
X.shape # TF-IDF 행렬의 크기 확인

(11314, 1000)

### 토픽 모델링(Topic Modeling)

In [21]:
from sklearn.decomposition import TruncatedSVD
svd_model = TruncatedSVD(n_components=20, algorithm='randomized', n_iter=100, random_state=122)
svd_model.fit(X)
len(svd_model.components_)   # 뉴스그룹 데이터가 20개의 카테고리를 갖고있었기 때문에, 20개의 토픽을 가졌다고 가정하고 토픽 모델링

20

In [22]:
np.shape(svd_model.components_)  # LSA에서 V.T에 해당, 토픽의 수 t(20), 단어수(1000)

(20, 1000)

In [23]:
terms = vectorizer.get_feature_names() # 단어 집합. 1,000개의 단어가 저장됨.

# 값이 큰 5개의 값을 찾아서 단어로 출력
def get_topics(components, feature_names, n=5):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(5)) for i in topic.argsort()[:-n - 1:-1]])

get_topics(svd_model.components_,terms)

# 각 20개의 행의 각 1,000개의 열 중 가장 값이 큰 5개의 값을 찾아서 단어로 출력

Topic 1: [('like', 0.21378), ('know', 0.20032), ('people', 0.19317), ('think', 0.17807), ('good', 0.15105)]
Topic 2: [('thanks', 0.3292), ('windows', 0.29094), ('card', 0.18016), ('drive', 0.1739), ('mail', 0.15126)]
Topic 3: [('game', 0.37161), ('team', 0.32553), ('year', 0.28204), ('games', 0.25416), ('season', 0.18464)]
Topic 4: [('drive', 0.52804), ('scsi', 0.20035), ('disk', 0.15514), ('hard', 0.15506), ('card', 0.14041)]
Topic 5: [('windows', 0.4052), ('file', 0.25609), ('window', 0.18057), ('files', 0.1619), ('program', 0.1401)]
Topic 6: [('chip', 0.16138), ('government', 0.16026), ('mail', 0.15625), ('space', 0.1505), ('information', 0.13575)]
Topic 7: [('like', 0.67109), ('bike', 0.14268), ('know', 0.11428), ('chip', 0.11058), ('sounds', 0.10398)]
Topic 8: [('card', 0.45115), ('sale', 0.21607), ('video', 0.21391), ('monitor', 0.14913), ('offer', 0.14872)]
Topic 9: [('know', 0.44979), ('card', 0.35491), ('chip', 0.17206), ('video', 0.15184), ('government', 0.15053)]
Topic 10: [

In [24]:
documents[20]

"\n\n\n\n\tI'd like to see this info as well.  As for wavelength, I think\nyou're primarily going to find two - 880 nM +/- a bit, and/or 950 nM\n+/- a bit.  Usually it is about 10 nM either way.  The two most common\nI have seen were 880 and 950 but I have also heard of 890 and 940.\nI'm not sure that the 10 nM one way or another will make a great deal of\ndifference.\n\n\tAnother suggestion - find a brand of TV that uses an IR remote,\nand go look at the SAMS photofact for it.  You can often find some very\ndetailed schematics and parts list for not only the receiver but the\ntransmitter as well, including carrier freq. specs. and tone decoding\nspecs. if the system uses that."

In [25]:
# LSA의 단점 : SVD의 특성상 이미 계산된 LSA에 새로운 데이터를 추가하여 계산하려고하면 보통 처음부터 다시 계산해야 한다. 
# 즉, 새로운 정보에 대해 업데이트가 어렵다. 

## [3] 잠재 디리클레 할당(Latent Dirichlet Allocation, LDA)]
토픽 모델링의 대표적인 알고리즘, LDA는 문서들은 토픽들의 혼합으로 구성되어져 있으며, 토픽들은 확률 분포에 기반하여 단어들을 생성한다고 가정한다.<br>
데이터가 주어지면, LDA는 문서가 생성되던 과정을 역추적 한다. <br>
LSA : DTM을 차원 축소 하여 축소 차원에서 근접 단어들을 토픽으로 묶는다.  <br>
LDA : 단어가 특정 토픽에 존재할 확률과 문서에 특정 토픽이 존재할 확률을 결합확률로 추정하여 토픽을 추출한다.

In [26]:
print(tokenized_doc.shape)
tokenized_doc[:5]

(11314,)


0    [well, sure, story, seem, biased, disagree, st...
1    [yeah, expect, people, read, actually, accept,...
2    [although, realize, principle, strongest, poin...
3    [notwithstanding, legitimate, fuss, proposal, ...
4    [well, change, scoring, playoff, pool, unfortu...
Name: clean_doc, dtype: object

In [27]:
# ! pip install gensim
from gensim import corpora
dictionary = corpora.Dictionary(tokenized_doc)
corpus = [dictionary.doc2bow(text) for text in tokenized_doc]
print(corpus[1]) # 수행된 결과에서 두번째 뉴스 출력. 첫번째 문서의 인덱스는 0

[(52, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 2), (67, 1), (68, 1), (69, 1), (70, 1), (71, 2), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 2), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 2), (86, 1), (87, 1), (88, 1), (89, 1)]


In [28]:
print(dictionary[66])

faith


In [29]:
len(dictionary)

64281

#### LDA 모델 훈련시키기

In [30]:
import gensim
NUM_TOPICS = 20    #20개의 토픽, k=20
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15) #  passes는 알고리즘의 동작 횟수
topics = ldamodel.print_topics(num_words=4)   #  4개의 단어만 출력 
for topic in topics:
    print(topic)
# 각 단어 앞에 붙은 수치는 단어의 해당 토픽에 대한 기여도    

(0, '0.021*"game" + 0.018*"team" + 0.014*"year" + 0.014*"games"')
(1, '0.015*"president" + 0.008*"administration" + 0.007*"technology" + 0.006*"national"')
(2, '0.017*"water" + 0.012*"ground" + 0.012*"cover" + 0.011*"henrik"')
(3, '0.008*"drive" + 0.008*"system" + 0.008*"like" + 0.006*"also"')
(4, '0.018*"simms" + 0.012*"helmet" + 0.010*"macs" + 0.010*"slots"')
(5, '0.019*"uuencode" + 0.017*"postscript" + 0.014*"version" + 0.009*"rockefeller"')
(6, '0.007*"david" + 0.006*"judas" + 0.005*"picture" + 0.005*"article"')
(7, '0.015*"available" + 0.011*"information" + 0.011*"data" + 0.009*"also"')
(8, '0.012*"know" + 0.011*"said" + 0.010*"people" + 0.010*"like"')
(9, '0.091*"space" + 0.032*"nasa" + 0.020*"launch" + 0.018*"earth"')
(10, '0.008*"time" + 0.008*"color" + 0.007*"image" + 0.007*"jpeg"')
(11, '0.010*"would" + 0.009*"jesus" + 0.008*"people" + 0.007*"believe"')
(12, '0.038*"jews" + 0.036*"turkish" + 0.028*"armenians" + 0.023*"turkey"')
(13, '0.011*"mask" + 0.009*"sweden" + 0.008*"cip

In [31]:
print(ldamodel.print_topics()) # 10개 단어를 출력

[(0, '0.021*"game" + 0.018*"team" + 0.014*"year" + 0.014*"games" + 0.013*"play" + 0.011*"season" + 0.009*"players" + 0.009*"hockey" + 0.009*"league" + 0.008*"last"'), (1, '0.015*"president" + 0.008*"administration" + 0.007*"technology" + 0.006*"national" + 0.006*"jobs" + 0.006*"government" + 0.006*"program" + 0.005*"congress" + 0.005*"going" + 0.005*"house"'), (2, '0.017*"water" + 0.012*"ground" + 0.012*"cover" + 0.011*"henrik" + 0.009*"wire" + 0.009*"neutral" + 0.007*"nist" + 0.006*"ncsl" + 0.006*"copies" + 0.005*"black"'), (3, '0.008*"drive" + 0.008*"system" + 0.008*"like" + 0.006*"also" + 0.006*"used" + 0.006*"would" + 0.005*"card" + 0.005*"good" + 0.005*"disk" + 0.005*"work"'), (4, '0.018*"simms" + 0.012*"helmet" + 0.010*"macs" + 0.010*"slots" + 0.010*"vram" + 0.008*"easter" + 0.008*"accelerator" + 0.007*"simm" + 0.007*"floppies" + 0.007*"uiuc"'), (5, '0.019*"uuencode" + 0.017*"postscript" + 0.014*"version" + 0.009*"rockefeller" + 0.008*"patch" + 0.006*"math" + 0.006*"courses" + 0.

#### 시각화

In [32]:
! pip install pyLDAvis

You should consider upgrading via the 'C:\ProgramData\Anaconda3\envs\tf20\python.exe -m pip install --upgrade pip' command.


In [33]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary)
pyLDAvis.display(vis)
# LDA 시각화에서는 토픽의 번호가 1부터 시작하므로 각 토픽 번호는  +1이 된 값인 1~20까지의 값을 가진다

####  문서 별 토픽 분포 보기

In [34]:
for i, topic_list in enumerate(ldamodel[corpus]):
    if i==5:
        break
    print(i,'번째 문서의 topic 비율은',topic_list)
# (숫자, 확률)은 각각 토픽 번호와 해당 토픽이 해당 문서에서 차지하는 분포도를 의미

0 번째 문서의 topic 비율은 [(1, 0.028266832), (6, 0.18351762), (12, 0.09420056), (16, 0.53564334), (18, 0.14605945)]
1 번째 문서의 topic 비율은 [(0, 0.09395123), (2, 0.027613647), (10, 0.26504236), (11, 0.59232104)]
2 번째 문서의 topic 비율은 [(0, 0.1383071), (1, 0.061532333), (8, 0.10842727), (16, 0.61880314), (18, 0.06079212)]
3 번째 문서의 topic 비율은 [(1, 0.09761796), (3, 0.38216162), (8, 0.06663931), (9, 0.018439313), (14, 0.14095843), (16, 0.2832067)]
4 번째 문서의 topic 비율은 [(0, 0.46968487), (8, 0.13075742), (17, 0.2552859), (19, 0.1146333)]


In [35]:
def make_topictable_per_doc(ldamodel, corpus):
    topic_table = pd.DataFrame()

    # 몇 번째 문서인지를 의미하는 문서 번호와 해당 문서의 토픽 비중을 한 줄씩 꺼내온다.
    for i, topic_list in enumerate(ldamodel[corpus]):
        doc = topic_list[0] if ldamodel.per_word_topics else topic_list            
        doc = sorted(doc, key=lambda x: (x[1]), reverse=True)
        # 각 문서에 대해서 비중이 높은 토픽순으로 토픽을 정렬한다.
        # EX) 정렬 전 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (10번 토픽, 5%), (12번 토픽, 21.5%), 
        # Ex) 정렬 후 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (12번 토픽, 21.5%), (10번 토픽, 5%)
        # 48 > 25 > 21 > 5 순으로 정렬이 된 것.

        # 모든 문서에 대해서 각각 아래를 수행
        for j, (topic_num, prop_topic) in enumerate(doc): #  몇 번 토픽인지와 비중을 나눠서 저장한다.
            if j == 0:  # 정렬을 한 상태이므로 가장 앞에 있는 것이 가장 비중이 높은 토픽
                topic_table = topic_table.append(pd.Series([int(topic_num), round(prop_topic,4), topic_list]), ignore_index=True)
                # 가장 비중이 높은 토픽과, 가장 비중이 높은 토픽의 비중과, 전체 토픽의 비중을 저장한다.
            else:
                break
    return(topic_table)

In [36]:
topictable = make_topictable_per_doc(ldamodel, corpus)
topictable = topictable.reset_index() # 문서 번호을 의미하는 열(column)로 사용하기 위해서 인덱스 열을 하나 더 만든다.
topictable.columns = ['문서 번호', '가장 비중이 높은 토픽', '가장 높은 토픽의 비중', '각 토픽의 비중']
topictable[:10]

,문서 번호,가장 비중이 높은 토픽,가장 높은 토픽의 비중,각 토픽의 비중
0,0,16.0,0.5357,"[(1, 0.028271697), (6, 0.18350112), (12, 0.094..."
1,1,11.0,0.5923,"[(0, 0.09397328), (2, 0.027613651), (10, 0.265..."
2,2,16.0,0.6188,"[(0, 0.13830611), (1, 0.061530042), (8, 0.1084..."
3,3,3.0,0.3822,"[(1, 0.09762002), (3, 0.3821705), (8, 0.066627..."
4,4,0.0,0.4697,"[(0, 0.46967965), (8, 0.13076316), (17, 0.2552..."
5,5,10.0,0.4428,"[(6, 0.07465813), (10, 0.44282052), (11, 0.354..."
6,6,3.0,0.8603,"[(3, 0.8603361), (14, 0.016978819), (19, 0.110..."
7,7,16.0,0.4521,"[(0, 0.042559847), (1, 0.15055169), (3, 0.0338..."
8,8,10.0,0.7477,"[(10, 0.7477038), (11, 0.2257853)]"
9,9,3.0,0.4526,"[(0, 0.014965166), (1, 0.10357825), (3, 0.4526..."


###  잠재 디리클레 할당(LDA) 구현 : 뉴스 기사 헤드라인 

In [37]:
import pandas as pd
import urllib.request
urllib.request.urlretrieve("https://raw.githubusercontent.com/franciscadias/data/master/abcnews-date-text.csv", filename="abcnews-date-text.csv")
data = pd.read_csv('abcnews-date-text.csv', error_bad_lines=False)
# 약 15년 동안 발행되었던 뉴스 기사 제목을 모아놓은 영어 데이터

In [38]:
print(len(data))

1082168


In [39]:
print(data.head(5))

   publish_date                                      headline_text
0      20030219  aba decides against community broadcasting lic...
1      20030219     act fire witnesses must be aware of defamation
2      20030219     a g calls for infrastructure protection summit
3      20030219           air nz staff in aust strike for pay rise
4      20030219      air nz strike to affect australian travellers


In [40]:
text = data[['headline_text']]
text.head(5)

,headline_text
0,aba decides against community broadcasting lic...
1,act fire witnesses must be aware of defamation
2,a g calls for infrastructure protection summit
3,air nz staff in aust strike for pay rise
4,air nz strike to affect australian travellers


In [42]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\IPA(백찬휘)\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [43]:
# 텍스트 전처리
import nltk
text['headline_text'] = text.apply(lambda row: nltk.word_tokenize(row['headline_text']), axis=1) 

C:\ProgramData\Anaconda3\envs\tf20\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [44]:
print(text.head(5))

                                       headline_text
0  [aba, decides, against, community, broadcastin...
1  [act, fire, witnesses, must, be, aware, of, de...
2  [a, g, calls, for, infrastructure, protection,...
3  [air, nz, staff, in, aust, strike, for, pay, r...
4  [air, nz, strike, to, affect, australian, trav...


In [45]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
text['headline_text'] = text['headline_text'].apply(lambda x: [word for word in x if word not in (stop)]) # 불용어 제거

C:\ProgramData\Anaconda3\envs\tf20\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [46]:
print(text.head(5))

                                       headline_text
0   [aba, decides, community, broadcasting, licence]
1    [act, fire, witnesses, must, aware, defamation]
2     [g, calls, infrastructure, protection, summit]
3          [air, nz, staff, aust, strike, pay, rise]
4  [air, nz, strike, affect, australian, travellers]


In [47]:
from nltk.stem import WordNetLemmatizer
text['headline_text'] = text['headline_text'].apply(lambda x: [WordNetLemmatizer().lemmatize(word, pos='v') for word in x]) # 표제어 추출
print(text.head(5))

                                       headline_text
0       [aba, decide, community, broadcast, licence]
1      [act, fire, witness, must, aware, defamation]
2      [g, call, infrastructure, protection, summit]
3          [air, nz, staff, aust, strike, pay, rise]
4  [air, nz, strike, affect, australian, travellers]


C:\ProgramData\Anaconda3\envs\tf20\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [48]:
tokenized_doc = text['headline_text'].apply(lambda x: [word for word in x if len(word) > 3])  # 길이가 3이하인 단어에 제거
print(tokenized_doc[:5])

0       [decide, community, broadcast, licence]
1      [fire, witness, must, aware, defamation]
2    [call, infrastructure, protection, summit]
3                   [staff, aust, strike, rise]
4      [strike, affect, australian, travellers]
Name: headline_text, dtype: object


#### TF-IDF 행렬 만들기

In [49]:
# 역토큰화 (토큰화 작업을 되돌림)
detokenized_doc = []
for i in range(len(text)):
    t = ' '.join(tokenized_doc[i])
    detokenized_doc.append(t)

text['headline_text'] = detokenized_doc # 다시 text['headline_text']에 재저장

C:\ProgramData\Anaconda3\envs\tf20\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [50]:
text['headline_text'][:5]

0       decide community broadcast licence
1       fire witness must aware defamation
2    call infrastructure protection summit
3                   staff aust strike rise
4      strike affect australian travellers
Name: headline_text, dtype: object

In [51]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(stop_words='english', 
max_features= 1000) # 상위 1,000개의 단어를 보존 
X = vectorizer.fit_transform(text['headline_text'])
X.shape # TF-IDF 행렬의 크기 확인

(1082168, 1000)

####  토픽 모델링

In [52]:
from sklearn.decomposition import LatentDirichletAllocation
lda_model=LatentDirichletAllocation(n_components=10,learning_method='online',random_state=777,max_iter=1)

In [53]:
lda_top = lda_model.fit_transform(X)

In [54]:
print(lda_model.components_)
print(lda_model.components_.shape) 

[[1.00001533e-01 1.00001269e-01 1.00004179e-01 ... 1.00006124e-01
  1.00003111e-01 1.00003064e-01]
 [1.00001199e-01 1.13513398e+03 3.50170830e+03 ... 1.00009349e-01
  1.00001896e-01 1.00002937e-01]
 [1.00001811e-01 1.00001151e-01 1.00003566e-01 ... 1.00002693e-01
  1.00002061e-01 7.53381835e+02]
 ...
 [1.00001065e-01 1.00001689e-01 1.00003278e-01 ... 1.00006721e-01
  1.00004902e-01 1.00004759e-01]
 [1.00002401e-01 1.00000732e-01 1.00002989e-01 ... 1.00003517e-01
  1.00001428e-01 1.00005266e-01]
 [1.00003427e-01 1.00002313e-01 1.00007340e-01 ... 1.00003732e-01
  1.00001207e-01 1.00005153e-01]]
(10, 1000)


In [55]:
terms = vectorizer.get_feature_names() # 단어 집합. 1,000개의 단어가 저장됨.

def get_topics(components, feature_names, n=5):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(2)) for i in topic.argsort()[:-n - 1:-1]])
        
get_topics(lda_model.components_,terms)

Topic 1: [('government', 8725.19), ('sydney', 8393.29), ('queensland', 7720.12), ('change', 5874.27), ('home', 5674.38)]
Topic 2: [('australia', 13691.08), ('australian', 11088.95), ('melbourne', 7528.43), ('world', 6707.7), ('south', 6677.03)]
Topic 3: [('death', 5935.06), ('interview', 5924.98), ('kill', 5851.6), ('jail', 4632.85), ('life', 4275.27)]
Topic 4: [('house', 6113.49), ('2016', 5488.19), ('state', 4923.41), ('brisbane', 4857.21), ('tasmania', 4610.97)]
Topic 5: [('court', 7542.74), ('attack', 6959.64), ('open', 5663.0), ('face', 5193.63), ('warn', 5115.01)]
Topic 6: [('market', 5545.86), ('rural', 5502.89), ('plan', 4828.71), ('indigenous', 4223.4), ('power', 3968.26)]
Topic 7: [('charge', 8428.8), ('election', 7561.63), ('adelaide', 6758.36), ('make', 5658.99), ('test', 5062.69)]
Topic 8: [('police', 12092.44), ('crash', 5281.14), ('drug', 4290.87), ('beat', 3257.58), ('rise', 2934.92)]
Topic 9: [('fund', 4693.03), ('labor', 4047.69), ('national', 4038.68), ('council', 40